<a href="https://colab.research.google.com/github/TryAnotherName/course_test/blob/main/final_work_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Раскомментить, если либы не установлены

# pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=2bb1229865f1f94d881f7a329132eb6acc095008742b46bb2348aed289db5877
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [44]:
# Импортим нужные функции
from pyspark.sql import SparkSession
from pyspark.sql.functions import count
from pyspark.sql.functions import col
from pyspark.sql.functions import sum as summary
from pyspark.sql.functions import to_date

In [14]:
# Создание SparkSession
spark = SparkSession.builder.appName("Read CSV").getOrCreate()

# Чтение CSV-файла, вывод схемы и пример данных
df = spark.read.csv("/content/web_server_logs.csv", header=True, inferSchema=True)
df.printSchema()
df.show()
df.createOrReplaceTempView("site_logs")

root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)

+---------------+--------------------+------+--------------------+-------------+-------------+
|             ip|           timestamp|method|                 url|response_code|response_size|
+---------------+--------------------+------+--------------------+-------------+-------------+
|  195.20.196.25|2024-08-23 04:21:...|DELETE|tags/wp-content/c...|          301|         2993|
|182.212.187.100|2024-08-25 06:34:...|   GET|          tag/search|          404|          579|
|146.131.177.176|2024-01-13 05:58:...|  POST|      categories/tag|          404|         5922|
| 104.181.93.163|2024-04-15 12:45:...|DELETE|               posts|          301|         4479|
| 92.151.163.149|2024-04-22 07:34:...|DELETE|          categories|          200|      

# Первое задание
## Исполнение через DataFrame API


In [21]:
df.groupBy('ip') \
.agg(count('*').alias('request_count')) \
.orderBy('request_count', ascending=False) \
.show(10)

+--------------+-------------+
|            ip|request_count|
+--------------+-------------+
|186.183.22.102|            2|
|22.144.171.196|            1|
| 141.200.62.78|            1|
|   87.223.2.10|            1|
| 72.187.183.18|            1|
| 22.117.39.167|            1|
|102.215.132.57|            1|
| 182.114.72.23|            1|
|  44.39.135.82|            1|
| 109.99.252.58|            1|
+--------------+-------------+
only showing top 10 rows



## Исполнение через SQL API

In [51]:
first_quest = spark.sql("""
SELECT ip, COUNT(*) request_count
FROM site_logs
GROUP BY ip
ORDER BY request_count DESC
LIMIT 10
""")

# Второе задание
## Исполнение через DataFrame API

In [24]:
df.groupBy('method') \
.agg(count('*').alias('method_count')) \
.show()

+------+------------+
|method|method_count|
+------+------------+
|  POST|       25296|
|DELETE|       24854|
|   PUT|       24927|
|   GET|       24923|
+------+------------+



## Исполнение через SQL API

In [52]:
second_quest = spark.sql("""
SELECT method, COUNT(*) method_count
FROM site_logs
GROUP BY method
""")

# Третье задание
## Исполнение через DataFrame API

In [37]:
df.filter(col('response_code') == 404) \
.count()

25142

## Исполнение через SQL API

In [53]:
third_quest = spark.sql("""
SELECT COUNT(*) response_count
FROM site_logs
WHERE response_code = 404
""")

# Четвертое задание
## Исполнение через DataFrame API

In [46]:
df.groupBy(to_date('timestamp').alias('date')) \
.agg(summary('response_size').alias('total_response_size')) \
.orderBy('date') \
.show()

+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-01-01|            1835502|
|2024-01-02|            1853829|
|2024-01-03|            1848673|
|2024-01-04|            1909639|
|2024-01-05|            1844522|
|2024-01-06|            1861087|
|2024-01-07|            2032972|
|2024-01-08|            1711595|
|2024-01-09|            1790842|
|2024-01-10|            1903851|
|2024-01-11|            1857280|
|2024-01-12|            1734784|
|2024-01-13|            1896822|
|2024-01-14|            1938702|
|2024-01-15|            1736091|
|2024-01-16|            1847198|
|2024-01-17|            1811768|
|2024-01-18|            1816352|
|2024-01-19|            1707781|
|2024-01-20|            1976929|
+----------+-------------------+
only showing top 20 rows



## Исполнение через SQL API

In [54]:
fourth_quest = spark.sql("""
SELECT DATE(timestamp) date, SUM(response_size) total_response_size
FROM site_logs
GROUP BY date
ORDER BY date
""")

# Вывод итоговых результатов в требуемом формате

In [67]:
print('Top 10 active IP addresses:')
first_quest.show()
print('Request count by HTTP method:')
second_quest.show()
print('Number of 404 response codes:', third_quest.collect()[0][0])
print('Total response size by day:')
fourth_quest.show()

Top 10 active IP addresses:
+--------------+-------------+
|            ip|request_count|
+--------------+-------------+
|186.183.22.102|            2|
|22.144.171.196|            1|
| 141.200.62.78|            1|
|   87.223.2.10|            1|
| 72.187.183.18|            1|
| 22.117.39.167|            1|
|102.215.132.57|            1|
| 182.114.72.23|            1|
|  44.39.135.82|            1|
| 109.99.252.58|            1|
+--------------+-------------+

Request count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|  POST|       25296|
|DELETE|       24854|
|   PUT|       24927|
|   GET|       24923|
+------+------------+

Number of 404 response codes: 25142
Total response size by day:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-01-01|            1835502|
|2024-01-02|            1853829|
|2024-01-03|            1848673|
|2024-01-04|            1909639|
|2024-01-05|            1844522|
|2

In [ ]:
spark.stop()